In [237]:
import requests
import re
import time
import string
import random
import math
import pandas as pd
from urllib.parse import urlparse
from collections import Counter
from bs4 import BeautifulSoup
from pymystem3 import Mystem

# Задание №1

In [238]:
def crawler(urls):
    links_tree = set()
    for url in urls:
        if len(texts) >= 10:
            return
    
        if url in visited:
            continue
        visited.add(url)
            
        try:
            #получаем контент страницы и декодируем его
            html = requests.get(url).content.decode()
        except:
            time.sleep(1)
            continue
        
        #парсинг страницы
        soup = BeautifulSoup(html)
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text(' ').strip().replace('\xa0', ' ').replace('\n', '').replace('\r', '').replace('\t', '')
        text = re.sub(' +', ' ', text)
    
        if len(text.split(' ')) >= 10:
            texts[url] = text
            
        parsed = urlparse(url)
        base = f"{parsed.scheme}://{parsed.netloc}"    
        links = re.findall('''<a\s+(?:[^>]*?\s+)?href="([^"]*)"''', html) 
        
        for i, link in enumerate(links):    
            if not urlparse(link).netloc:    
                link_with_base = base + link    
                links[i] = link_with_base
                
        links_tree.update(set(filter(lambda x: 'mailto' not in x, links)))
    
    crawler(links_tree)

In [239]:
texts = {}
visited = set()
start = 'https://m.habr.com/ru/all/'

crawler([start])

In [240]:
index = {i:k for i, k in enumerate(texts)} 

In [241]:
with open('index.txt', 'w') as f:
    for i in index:
        f.write(f"{i}: {index[i]}\n")

# Задание №2

In [242]:
from openpyxl import load_workbook
import pandas as pd
import os, pymorphy2

In [243]:
words_all = {}
lemma_words = {}
lemma_normal_forms = {}

In [244]:
def get_sparsed_list(word_list):
    # получить список всех словоформ
    morph, lst = pymorphy2.MorphAnalyzer(), []
    for word in word_list:
        lemma = ""
        for x in morph.parse(word)[0].lexeme:
            lemma+=x.word + " "
        lst.append(f"{word} {lemma}\n")
    return lst

In [246]:
texts_lemmatized = {}

for url in texts:
    text = texts[url]
    # оставляем только слова
    text = re.sub(r'[^a-zA-Zа-яА-Я]+', ' ', text)
   
    words = text.split(' ')
    words_all[url]=words
    
    texts_lemmatized[url]=get_sparsed_list(words_all[url])  

In [247]:
with open('words.txt', 'w') as f:
    for url in words_all:
        for i, score in enumerate(words_all[url]):
            f.write(f"{score}\n")

In [248]:
with open('lemma_words.txt', 'w') as f:
    for url in texts_lemmatized:
        for i, score in enumerate(texts_lemmatized[url]):
            f.write(f"{score}\n")

# Задание №3

In [249]:
docs = [index[k] for k in index]

In [250]:
reverse_index = {}

for url in texts_lemmatized:
    for word in texts_lemmatized[url]:
        for word1 in word.split():
            if not word1 in reverse_index:
                 # если слово не входит в reverse_index, то добавляем его туда
                reverse_index[word1] = set([url])
            else:
                reverse_index[word1].update([url])

In [251]:
logic = {
    '&': lambda x,y: x & y,
    '|': lambda x,y: x | y,
    '!': lambda x: set([doc for doc in docs if doc not in x])
}

def find(query):
    query = query.replace('(', ' ( ').replace(')', ' ) ').replace('!','! ').split()
    
    indexes_words = [i for i,x in enumerate(query) if not x in ['!', '&', '|']]
    for i in indexes_words:
        query[i] = reverse_index[query[i]]
    
    indexes_not = [i for i,x in enumerate(query) if x == '!']
    for i in indexes_not:
        x = query[i+1]
        query[i+1] = logic['!'](x)
        query.pop(i)
    
    indexes_and = [i for i,x in enumerate(query) if x == '&']
    for i in indexes_and:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['&'](x,y)
        query.pop(i)
        query.pop(i)
    
    indexes_or = [i for i,x in enumerate(query) if x == '|']
    for i in indexes_or:
        x = query[i-1]
        y = query[i+1]
        query[i-1] = logic['|'](x,y)
        query.pop(i)
        query.pop(i)
    
    return query[0]

In [252]:
x1 = random.choice(list(reverse_index.keys()))
x2 = random.choice(list(reverse_index.keys()))
x3 = random.choice(list(reverse_index.keys()))

In [253]:
query = f'{x1} & !{x3}'
print(query)

фанатился & !попадаешься


In [254]:
find(query)

{'https://m.habr.com/ru/news/t/546914/comments/'}

# Задание №4

In [255]:
tfs = {}

for word in reverse_index:
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs:
            tfs[word] = {}
        tfs[word][doc] = tf

In [263]:
with open('task4_tfs.txt', 'w') as f:
    for word in reverse_index:
        for doc in docs:
            f.write(f"{word} {doc} {tfs[word][doc]}\n")

In [264]:
pd.DataFrame(tfs).T

,https://m.habr.com/ru/all/,https://m.habr.com/ru/post/546928/comments/,https://m.habr.com/ru/news/t/546914/comments/,https://tmtm.ru/services/content/,https://m.habr.com/ru/post/546898/,https://m.habr.com/ru/news/,https://career.habr.com/courses/186?utm_campaign=courses_postlist&amp;utm_content=task&amp;utm_medium=habr_block&amp;utm_source=habr_mob,https://m.habr.com/ru/post/546946/,https://m.habr.com/ru/post/546946/comments/,https://m.habr.com/ru/users/Nikitin_ant/
Все,0.007168,0.002395,0.001931,0.0,0.000521,0.003836,0.005435,0.00128,0.001565,0.008264
всё,0.000000,0.001198,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
публикации,0.001195,0.000000,0.000000,0.0,0.001043,0.000000,0.002717,0.00128,0.000000,0.000000
публикация,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
публикацию,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
приглашениями,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
приглашеньями,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
приглашениях,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
приглашеньях,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000


In [265]:
idfs = {}

for word in reverse_index:
    idfs[word] = math.log(len(docs) / len(reverse_index[word]))

In [266]:
with open('task4_idfs.txt', 'w') as f:
    for word in reverse_index:
        f.write(f"{word} {idfs[word]}\n")

In [267]:
pd.DataFrame(idfs,index=['']).T

,
Все,0.105361
всё,0.105361
публикации,0.000000
публикация,0.000000
публикацию,0.000000
...,...
приглашениями,2.302585
приглашеньями,2.302585
приглашениях,2.302585
приглашеньях,2.302585


In [268]:
tfidfs = {}

for word in tfs:
    for doc in tfs[word]:
        tfidf = tfs[word][doc] * idfs[word]
        if not word in tfidfs:
            tfidfs[word] = {}
        tfidfs[word][doc] = tfidf

In [269]:
with open('task4_tfidfs.txt', 'w') as f:
    for word in reverse_index:
        for doc in docs:
            f.write(f"{word} {doc} {tfidfs[word][doc]}\n")

In [270]:
pd.DataFrame(tfidfs).T

,https://m.habr.com/ru/all/,https://m.habr.com/ru/post/546928/comments/,https://m.habr.com/ru/news/t/546914/comments/,https://tmtm.ru/services/content/,https://m.habr.com/ru/post/546898/,https://m.habr.com/ru/news/,https://career.habr.com/courses/186?utm_campaign=courses_postlist&amp;utm_content=task&amp;utm_medium=habr_block&amp;utm_source=habr_mob,https://m.habr.com/ru/post/546946/,https://m.habr.com/ru/post/546946/comments/,https://m.habr.com/ru/users/Nikitin_ant/
Все,0.000755,0.000252,0.000203,0.0,0.000055,0.000404,0.000573,0.000135,0.000165,0.000871
всё,0.000000,0.000126,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
публикации,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
публикация,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
публикацию,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
приглашениями,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
приглашеньями,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
приглашениях,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
приглашеньях,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Задание №5

In [271]:
docs_length = {}
for row in pd.DataFrame(tfidfs).iterrows():
    sum_len = 0
    for word in row[1]:
        sum_len += word**2
    docs_length[row[0]] = math.sqrt(sum_len)

In [272]:
lemmatizer = Mystem()

In [313]:
query = "все поток"
query = get_sparsed_list(query.split())
query = " ".join([line.rstrip() for line in query])

In [314]:
tfs_query = {}
for word in query.split():
    for doc in docs:
        tokens = texts[doc].split()
        tf = tokens.count(word) / len(tokens)
        if not word in tfs_query:
            tfs_query[word] = {}
        tfs_query[word][doc] = tf

In [315]:
idfs_query = {}

for word in query.split():
    idfs_query[word] = math.log(len(docs) / len(reverse_index[word]))

In [316]:
tfidfs_query = {}

for word in tfs_query:
    for doc in tfs_query[word]:
        tfidf = tfs_query[word][doc] * idfs_query[word]
        if not word in tfidfs_query:
            tfidfs_query[word] = {}
        tfidfs_query[word][doc] = tfidf

In [317]:
tfidfs_query_flat = pd.DataFrame(tfidfs_query).T.sum().to_dict()

In [318]:
sum_len = 0
for doc in tfidfs_query_flat:
    sum_len += tfidfs_query_flat[doc]**2
query_length = math.sqrt(sum_len)

In [319]:
cos_sim = {}

for doc in docs:
    my_list = list()
    for word in query.split(' '):
        my_list.append(((pd.Series(tfidfs_query_flat) * tfs[word][doc]).sum()) / (docs_length[doc] * query_length))
    cos_sim[doc]=max(my_list)

In [320]:
Counter(cos_sim).most_common(10)

[('https://m.habr.com/ru/users/Nikitin_ant/', 0.2268794520600359),
 ('https://m.habr.com/ru/all/', 0.16429687722129988),
 ('https://m.habr.com/ru/post/546946/', 0.16096336916517057),
 ('https://m.habr.com/ru/news/', 0.1175771420755635),
 ('https://m.habr.com/ru/post/546898/', 0.09182489778515969),
 ('https://m.habr.com/ru/news/t/546914/comments/', 0.08391460888439148),
 ('https://m.habr.com/ru/post/546946/comments/', 0.07681837843349418),
 ('https://m.habr.com/ru/post/546928/comments/', 0.052772110585167724),
 ('https://tmtm.ru/services/content/', 0.0),
 ('https://career.habr.com/courses/186?utm_campaign=courses_postlist&amp;utm_content=task&amp;utm_medium=habr_block&amp;utm_source=habr_mob',
  0.0)]